In [1]:
################################################################################################################################################################################
# Importamos las librerías necesarias y generamos conexión con Binance para descargar la lista de pares que binance maneja.
################################################################################################################################################################################
import pandas as pd
import numpy as np
from datetime import datetime
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from binance import Client
from IPython.display import clear_output

client = Client()

###################################
# Parámetros de estrategia.
###################################
smooth_interval = 20
smooth_exp = 3
std_mult = 0.25
pdi_mult = 15
mapd_mult = 25
sdi_mult = 50
k = 0

while True:
    clear_output(wait=True)
    ##################################################################################################################################
    # Descarga y limpieza de datos de X días atrás a hoy.
    ##################################################################################################################################
    recent_data = pd.read_csv('../../../../csvs/streaming/BTCUSDT.csv')
    recent_data.columns = ['Tiempo','Simbolo','Open']
    recent_data = recent_data[-60*5:]
    recent_data = recent_data.loc[:,['Tiempo','Open']]
    recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
    recent_data = recent_data.set_index('Tiempo')
    recent_data['Open'] = recent_data.Open.astype(float)
    recent_data = recent_data.reset_index()
    
    # if k == 0:
    #     print(f'Inicia el bot de trading con inicio en: {datetime.utcnow()}')
    #     k += 1

    ##################################################
    # Interpolación (suavizado)  del Precio (OPEN) con SciPy y obtención de derivadas con su respectiva interpolación.
    ##################################################
    recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Open,smooth_interval,smooth_exp)
    recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
    recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
    recent_data['MAPD'] = recent_data.PD_Interpolacion.rolling(window=15).mean()
    mp = recent_data.Open.mean()

    ######################################################################
    # Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
    ######################################################################
    # initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[:1].item()),'%Y-%m-%d %H:%M:%S')].index.item() + rango
    # final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S')].index.item() - rango

    abs_open_std = abs(np.std(recent_data.Open)*std_mult)
    pdi_cond = recent_data.PD_Interpolacion*pdi_mult
    sdi_cond = recent_data.SD_Interpolacion
    
    recent_data['sdi_updown'] = np.select(condlist=[(abs(sdi_cond*sdi_mult) > abs_open_std)],
                                                        choicelist=[1],
                                                        default=0)
    recent_data['pdi_updown'] = np.select(condlist=[(pdi_cond > abs_open_std),
                                                        (pdi_cond < -abs_open_std)],
                                                        choicelist=[1,-1],
                                                        default=0)
    recent_data['sell_buy_side'] = np.select(condlist=[sdi_cond > 0,
                                                        sdi_cond < 0],
                                                        choicelist=[1,-1],
                                                        default=0)
    
    
    recent_data['signal'] = 0
    recent_data.loc[(recent_data.sell_buy_side == 1) & (recent_data.sdi_updown == 1) & (recent_data.pdi_updown == 1) & (mp + recent_data.MAPD*mapd_mult > mp + abs_open_std*0.25),'signal'] = 1
    recent_data.loc[(recent_data.sell_buy_side == -1) & (recent_data.sdi_updown == 1) & (recent_data.pdi_updown == -1) & (mp + recent_data.MAPD*mapd_mult < mp + abs_open_std),'signal'] = -1

    ######################################################################
    # Ahora ya podemos visualizar los resultados con el siguiente código.
    ######################################################################
    plt.figure(figsize=(15,8))
    plt.plot(recent_data.Tiempo,recent_data.Open.multiply(1),color='y')
    plt.plot(recent_data.Tiempo,recent_data.PD_Interpolacion.multiply(pdi_mult)+mp,'purple')
    plt.plot(recent_data.Tiempo,recent_data.SD_Interpolacion.multiply(sdi_mult)+mp,color='k')
    plt.plot(recent_data.Tiempo,recent_data.MAPD.multiply(pdi_mult)+mp,'g')
    plt.scatter(recent_data.loc[recent_data.signal==1,'Tiempo'],recent_data.loc[recent_data.signal==1,'Open'],color='b')
    plt.scatter(recent_data.loc[recent_data.signal==-1,'Tiempo'],recent_data.loc[recent_data.signal==-1,'Open'],color='r')
    plt.grid(visible=True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y %H:%M'))
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=15))
    plt.gca().yaxis.set_major_locator(ticker.LinearLocator(40))
    plt.gcf().autofmt_xdate()
    plt.grid(which = 'minor')
    plt.xticks(rotation=75)
    plt.axhline(mp)
    plt.axhline(np.std(recent_data.Interpolacion)*std_mult + mp)
    plt.axhline(mp - np.std(recent_data.Interpolacion)*std_mult)
    plt.show()